Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve
import time

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s.' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip.


In [3]:
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return tf.compat.as_str(f.read(name))
  f.close()
  
text = read_data(filename)
print('Data size %d.' % len(text))

Data size 100000000.


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]  # The first 1000 chars are validation text
train_text = text[valid_size:]  # The rest are training text
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' ' = 27
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):  # ID of a char is serail number of a lower case char, starting from 1. ID of ' ' is 0, 'a' is 1, 'b' is 2, etc.
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)   # Return ID 0 for non-space, non-lower-case ascii char
    return 0
  
def id2char(dictid): # return ascii char based on char ID
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model. As this is a charactor prediction application, the training text is also the expected (label) text. The label text starts from the 2nd char while the training text starts from the first char.

Note that batch, whose shape is (batch_size, 27), is not continuous in text order between batches. A char in each batch is taken from text in different locations.

BatchGenerator::next() function returns (1+num_unrollings) batches, i.e. the shape of batches is (1+num_unrollings, batch_size, 27). Charactor in the same batch are in text sequence even across consecutive batches. This provides a continuous text within a batch for training. Learning (gradient descent) is executed every batches, i.e. every (1+num_unrollings, batch_size) charactors, i.e. each batch has (1+num_unrollings) continous text charactors.

As an example, the first batch in three batches, each of which is returned by BatchGenerator::next(), has the following texts:
'ons anarchi', 'ists advoca', 'ate social '
Note that the above text are continous except that there is a duplicate char at the end of a string and beginning of its next string.

batches2string() takes in batches parameter gernetated by BatchGenerator::next() and produces a list of batch_size strings . Each string in the returned list is restored to the text sequence of (1+num_unrollings) charactors.  

In [6]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    # segment size, each batch data is taken from a segement
    segment = self._text_size // batch_size  # text is split into batch_size(=64) segments
    # index into text. distance between successive index are segment
    self._cursor = [ offset * segment for offset in range(batch_size)] 
    self._last_batch = self._next_batch()
  
  def _next_batch(self):  # A batch (whose shape is (batch_size, 27)) is not continuous in text order.
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float) # 'batch' shape is (batch_size, 27).
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):  # The function returns 1+num_unrollings batches.
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):  # Skip num_unrollings batches
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]  # np.argmax() returns the indices of the maximum values along an axis.

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  #print(batches[0].shape)
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]  # This line restores text order in element of s
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

batches = train_batches.next()
print("shape of train batches:", len(batches), batches[0].shape)
print(batches2string(batches))
print(batches2string(train_batches.next()))

batches = valid_batches.next()
print("shape of validation batches:", len(batches), batches[0].shape)
print(batches2string(batches))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

shape of train batches: 11 (64, 27)
['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['i

In [7]:
# logprob() measures the similarity between predictions and labels. It computes the cross entropy between them. 
# The formular is the same as loss calculation.
# A single value is returned. The larger the returned value, the similarity is lower.
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  # np.log(): Natural logarithm, element-wise.
  # np.sum(): Sum of array elements over a given axis.The default, axis=None, will sum all of the elements of the input array.
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  # random.uniform() returns a random floating point number N such that a <= N <= b for a <= b
  r = random.uniform(0, 1) # return a random number between 0 and 1
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):  # Randomly produce a 1-hot encoding sample
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution(): # Generate a row of random probabilities
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  #print(np.sum(b, 1))
  return b/np.sum(b, 1)[:,None]

rd = random_distribution()
print(rd)
print(sample(rd))


[[ 0.0376637   0.05619306  0.02758124  0.00123307  0.03952761  0.00972838
   0.04319059  0.05483401  0.02648267  0.00758812  0.02998781  0.01516402
   0.04833469  0.07514684  0.08557528  0.00768075  0.08510656  0.02289891
   0.01614088  0.05216722  0.05668785  0.022882    0.01273959  0.05060542
   0.04600211  0.04006026  0.02879737]]
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
   0.  0.  0.  0.  0.  0.  0.  0.  0.]]


Simple LSTM Model.

In [8]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation. See Fig 10.6 in P373 and Fig 10.16 on P398 for this model.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    # tf.sigmoid() is an element wise sigmoid function. 
    # These gate's variables have a shape of (batch_size, num_nodes).
    # (batch_size,vocabulary_size)* (vocabulary_size, num_nodes) = (batch_size, num_nodes)
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)  
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    # update shape: (batch_size,vocabulary_size)* (vocabulary_size, num_nodes) = (batch_size, num_nodes)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    # state shape: (batch_size, num_nodes) * (batch_size, num_nodes) = (batch_size, num_nodes). 
    # Note this is an element wise multiplication.
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list() # create an empty list
  # train_data is a list with (num_unrollings + 1) items each of which is a [batch_size,vocabulary_size] matrix.
  # I.e. train_data shape is (num_unrollings+1, batch_size, vocabulary_size)
  for _ in range(num_unrollings + 1):
    train_data.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  # Note that train_inputs and train_labels have the same number of elements: num_unrollings
  train_inputs = train_data[:num_unrollings] # all train_data are train_inputs except the last item.
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output # Shape is (batch_size, num_nodes), initial value are all 0s.
  state = saved_state   # Shape is (batch_size, num_nodes), initial value are all 0s.
  for i in train_inputs:  # i is a (64, 27) matrix, i.e. (batch_size,vocabulary_size)
    # output and state are (batch_size, num_nodes) matrix
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # outputs are output of num_nodes LSTM cells, with output of num_unrollings times. 
  # logits is calculated based on outputs which is calculated by the loop above.
  # loss is caculated based on logits and train_labels, both of which contains data of a whole batch.
  # gradient descent is performed after a whole batch is calculated.
  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs,0), w, b) 
        # outputs is a list of num_unrollings elements each of which has shape(batch_size, num_nodes).
        # tf.concat(outputs, 0) shape is (num_unrollings*batch_size, num_nodes) = (640, 32)
        # Thus logits shape is (num_unrollings*batch_size, vocabulary_size) = (640, 27)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.concat(train_labels,0)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.5, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  # The following two variables are not parametered with "trainable=False" becasue no gradient descent 
  # is performed on them.
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [9]:
num_steps = 17001
summary_frequency = 200

start_time = time.time()
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):  # Construct train_data, which contains both train input data and train labels
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())  # feed is a random 1-hot encoding sample with shape (1, vocabulary_size)
          sentence = characters(feed)[0] # sentence is a char, such as 'h'
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed}) # See what is pridicated with 1 char as sample_input
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print("\nTime taken to step %d: %.2fs" % (step, time.time()-start_time))
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      sam_valid_text=""
      sam_prediction_text=""
      for _ in range(valid_size):
        b = valid_batches.next()
        sam_valid_text += characters(b[0])[0]
        predictions = sample_prediction.eval({sample_input: b[0]})
        sam_prediction_text += characters(predictions)[0]
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      if step % (summary_frequency * 10) == 0:
        print("validation text:", sam_valid_text[:20])
        print("predicted  text:", sam_prediction_text[:20])
      #print(b[0][0], characters(b[0])[0])
      #print(predictions[0], characters(predictions)[0])
      print('Validation set perplexity: %.2f\n' % float(np.exp(valid_logprob / valid_size)))
print("\nTime it takes to run the graph with %d:" % num_steps, time.time()-start_time)

Initialized
Average loss at step 0: 3.300726 learning rate: 10.000000
Minibatch perplexity: 27.13
wn  hhgkoozeomaetsapuye k ot kmto ia gdbnkp  hoxhvoohlcdkhhkounxmenxjfiikgi uq k
jgwixvupah s qe zj ql ssnadehdgarkmrqxroqrzhs jnapfpnihers a  yhe c p e eznrstet
ysvfe penzt fnuwtkssthpcuyts dorojiizeemxbttjhg mhlv leuhpmrgajiyrtearejqoylyi r
kxr sjss arxuhi knebecepdfnf f iamnahmcsthzn  rhoyvituh hmel q urpvpmpnmyo py nu
aynaa midbjrnofddd noe jm b bfa wbiwuboipmtpchmysftebhlxto qhmifhc vlidu a vvs e

Time taken to step 0: 64.08s
validation text: rchism originated as
predicted  text:                     
Validation set perplexity: 20.27

Average loss at step 200: 2.424789 learning rate: 10.000000
Minibatch perplexity: 8.52
Validation set perplexity: 8.39

Average loss at step 400: 2.049199 learning rate: 10.000000
Minibatch perplexity: 7.59
Validation set perplexity: 7.94

Average loss at step 600: 1.920290 learning rate: 10.000000
Minibatch perplexity: 6.33
Validation set perplexity: 7.0


---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [10]:
# Implementation for Problem 1, combined gate parameters

num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate, Forget gate, Memory cell, Output gate each having num_nodes columns in the variable. 
  # input, previous output, and bias.
  gx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes*4], -0.1, 0.1))
  gm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes*4], -0.1, 0.1))
  gb = tf.Variable(tf.zeros([1, num_nodes*4]))

  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation. See Fig 10.6 in P373 and Fig 10.16 on P398 for this model.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    # gates and update but sigmoid is not applied to gates yet.
    ig_fg_update_og = tf.matmul(i, gx) + tf.matmul(o, gm) + gb
    # state shape: (batch_size, num_nodes) * (batch_size, num_nodes) = (batch_size, num_nodes). 
    # Note this is an element wise multiplication.
    state = tf.sigmoid(ig_fg_update_og[:,num_nodes:2*num_nodes]) * state + tf.sigmoid(
        ig_fg_update_og[:,0:num_nodes]) * tf.tanh(ig_fg_update_og[:,num_nodes*2:num_nodes*3])
    return tf.sigmoid(ig_fg_update_og[:,num_nodes*3:]) * tf.tanh(state), state

  # Input data.
  train_data = list() # create an empty list
  # train_data is a list with (num_unrollings + 1) items each of which is a [batch_size,vocabulary_size] matrix.
  # I.e. train_data shape is (num_unrollings+1, batch_size, vocabulary_size)
  for _ in range(num_unrollings + 1):
    train_data.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  # Note that train_inputs and train_labels have the same number of elements: num_unrollings
  train_inputs = train_data[:num_unrollings] # all train_data are train_inputs except the last item.
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output # Shape is (batch_size, num_nodes), initial value are all 0s.
  state = saved_state   # Shape is (batch_size, num_nodes), initial value are all 0s.
  for i in train_inputs:  # i is a (64, 27) matrix, i.e. (batch_size,vocabulary_size)
    # output and state are (batch_size, num_nodes) matrix
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # outputs are output of num_nodes LSTM cells, with output of num_unrollings times. 
  # State saving across unrollings.
  # How does TF insure loss is calculated after all outputs elements are calculated?
  # Does it make sure all 11 loops to calculate Outputs are execuated before calculating logits?
  # logits are calculated after all outputs elements are collected.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs,0), w, b) 
        # outputs is a list of num_unrollings elements each of which has shape(batch_size, num_nodes).
        # tf.concat(outputs, 0) shape is (num_unrollings*batch_size, num_nodes) = (640, 32)
        # Thus logits shape is (num_unrollings*batch_size, vocabulary_size) = (640, 27)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.concat(train_labels,0)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.5, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  # The following two variables are not parametered with "trainable=False" becasue no gradient descent 
  # is performed on them.
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [11]:
# Exactly same training code as in example 
num_steps = 17001
summary_frequency = 200

start_time = time.time()
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())  # feed is a random 1-hot encoding sample with shape (1, vocabulary_size)
          sentence = characters(feed)[0] # sentence is a char, such as 'h'
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed}) # See what is pridicated with 1 char as sample_input
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print("\nTime taken to step %d: %.2fs" % (step, time.time()-start_time))
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      sam_valid_text=""
      sam_prediction_text=""
      for _ in range(valid_size):
        b = valid_batches.next()
        sam_valid_text += characters(b[0])[0]
        predictions = sample_prediction.eval({sample_input: b[0]})
        sam_prediction_text += characters(predictions)[0]
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      if step % (summary_frequency * 10) == 0:
        print("validation text:", sam_valid_text[:20])
        print("predicted  text:", sam_prediction_text[:20])
      #print(b[0][0], characters(b[0])[0])
      #print(predictions[0], characters(predictions)[0])
      print('Validation set perplexity: %.2f\n' % float(np.exp(valid_logprob / valid_size)))
print("\nTime it takes to run the graph with %d:" % num_steps, time.time()-start_time)

Initialized
Average loss at step 0: 3.298453 learning rate: 10.000000
Minibatch perplexity: 27.07
faf  jrrbbljj a  sditm  ttsxnevnc ra bzqataecxtxgwswum sewwlaeyh fnpnddlwxei  oe
qe na oi l tqvihogzg luj qxzqoued  erek oepin  wxenovbb bre cmmvqbiepbhmhmsci fw
jyi wywfqpw bhikjamimnkvf wukljqwfcaiwl j ekdh k  au swqo qsste ffwyerlrpbx a kc
dtaowepswsdc wwadzmsiejciauwqlytplwq qlrqrkgqb lx ms bfakjcin    meantwnm onnedm
vkmqw awecpveo  xerc  n lyybntzy esh aufccp itrlhcexuu raisc uetnxzrkptehsn z dy

Time taken to step 0: 0.40s
validation text: rchism originated as
predicted  text:                     
Validation set perplexity: 20.23

Average loss at step 200: 2.428590 learning rate: 10.000000
Minibatch perplexity: 8.87
Validation set perplexity: 8.75

Average loss at step 400: 2.066557 learning rate: 10.000000
Minibatch perplexity: 7.03
Validation set perplexity: 7.88

Average loss at step 600: 1.946584 learning rate: 10.000000
Minibatch perplexity: 7.40
Validation set perplexity: 7.00


Speed Improvement in Problem 1
---------------------------------
To run 17001 steps, the simple LSTM model takes 218s while the problem 1 model takes 127s, almost half the time is used by the problem 1 model. In simple LSTM model, the three gates and input update are calculated separately each of which uses theirown parameter matrixes. In Problem 1 model, the paramter matrixes for the four nodes are combined and the four nodes are calculated in one shot. This almost double the running speed of the model possibly due to less matrix multiplication and gradeient descent on less parameter matrixes.   

Problem 1+ : Will training be faster if all unrollings are fed into the model once?
--------------------------------------------------------------------------------------------
In either the Simple LSTM model or the Improved Large Matrix model (Probelm 1), the input data are fed in as 11 successive groups. 640 training data are fed into the learning model. Why don't we fed 640 training data as a large single input data? Remember that the model is RNN, earlier states and outputs are used to make predictions next. Each unrolling is an opportunity to update states. Feeding 640 data as a single input is like making batch size 640 and num_unrollings to be 1. The model should still work but the effect is unknown. There should also be speed improvement. How much? 


---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---